In [1]:
import subprocess

def cluster_elki(name, num_clusters):
    # Define parameters
    elki_jar = "elki-bundle-0.8.0.jar"
    data_file = f"output_analysis/output-{name}.csv"

    # Construct the ELKI command
    cmd = [
        "java", "-jar", elki_jar,
        "KDDCLIApplication",
        "-dbc.in", data_file,
        "-algorithm", "clustering.hierarchical.extraction.CutDendrogramByNumberOfClusters",
        "-algorithm", "Anderberg",
        "-algorithm.distancefunction", "CosineDistance",
        "-hierarchical.minclusters", str(num_clusters),
        "-resulthandler", "ResultWriter",
        "-out.gzip", "false",
        "-out", f"output_analysis/elki-TEST-{name}-{num_clusters}",
    ]

    # Execute the command
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    except subprocess.CalledProcessError as e:
        print("An error occurred:\n", e.stderr)

In [2]:
import os
import torch
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from training.inference import load_and_parse_audio

def process_song(song_path, chunking=True, chunk_size=256):
    """Preprocess one song (CPU only)."""
    chunks = load_and_parse_audio(song_path, convert=True, chunking=chunking, chunk_size=chunk_size)
    return song_path, chunks

def compute(model, name, batch_size=16, num_workers=4, chunking=True, chunk_size=256, start_partition=1):
    path = "E:\\SongsDataset\\songs\\"
    all_songs = os.listdir(path)
    if start_partition == 1:
        sign = 'w'
    else:
        sign = 'a'

    with open(name, sign, encoding="utf-8") as f, torch.no_grad(), ThreadPoolExecutor(max_workers=num_workers) as executor:
        # submit all preprocessing jobs
        partitions = 8
        num_songs = len(all_songs) // partitions
        for index in range(start_partition, partitions):
            futures = [executor.submit(process_song, os.path.join(path, song), chunking, chunk_size) for song in all_songs[(index - 1) * num_songs:index * num_songs]]

            # process results in batches of X
            for future in tqdm(as_completed(futures), total=len(futures)):
                song_path, chunks = future.result()
                run_batch(model, chunks, f, song_path)

def run_batch(model, chunks, file_handle, song_path):
    chunks = chunks.to("cuda")

    chunks = chunks.unsqueeze(1)
    latents = model(chunks)

    averages = latents.mean(dim=0).cpu().numpy()
    torch.cuda.empty_cache()
    line = " ".join([str(x) for x in averages]) + f" \"{os.path.basename(song_path)}\"\n"
    file_handle.write(line)

In [3]:
model = torch.load("E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\Myna-CLS-ALIBI\\Epoch-103.pt", weights_only=False)
model.mask_ratio = 0.0

In [ ]:
compute(model, "E:/Coding/SongAnalyzer/Analyzer/src/output_analysis/output-Myna-CLS-ALIBI-Chunking-256.csv", chunking=True, chunk_size=256)

 29%|██▉       | 144/496 [01:11<02:55,  2.01it/s]


In [4]:
cluster_elki("Myna-CLS-ALIBI-Chunking-256", 64)
cluster_elki("Myna-CLS-ALIBI-Chunking-256", 256)

In [4]:
compute(model, "E:/Coding/SongAnalyzer/Analyzer/src/output_analysis/output-Myna-CLS-ALIBI-Chunking-512.csv", chunking=True, chunk_size=512, batch_size=4, num_workers=3)
cluster_elki("Myna-CLS-ALIBI-Chunking-512", 64)
cluster_elki("Myna-CLS-ALIBI-Chunking-512", 256)

100%|██████████| 496/496 [03:33<00:00,  2.32it/s]


In [ ]:
compute(model, "E:/Coding/SongAnalyzer/Analyzer/src/output_analysis/output-Myna-CLS-ALIBI-Chunking-2048.csv", chunking=True, chunk_size=2048, batch_size=1, num_workers=1, start_partition=3)
cluster_elki("Myna-CLS-ALIBI-Chunking-2048", 64)
cluster_elki("Myna-CLS-ALIBI-Chunking-2048", 256)

  1%|          | 4/496 [00:07<17:19,  2.11s/it]

In [ ]:
compute(model, "E:/Coding/SongAnalyzer/Analyzer/src/output_analysis/output-Myna-CLS-ALIBI-No-Chunking.csv", chunking=False, chunk_size=131072, batch_size=1, num_workers=1)
cluster_elki("Myna-CLS-ALIBI-Chunking-No-Chunking", 64)
cluster_elki("Myna-CLS-ALIBI-Chunking-No-Chunking", 256)

  0%|          | 0/496 [00:11<?, ?it/s]
